In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

## What is ensemble modelling ?
It means combining different machine learning models to get a better prediction

### Ensemble Methods
1. Average (or blending)
2. Weighthed averaging
3. Conditional averaging
4. Bagging
5. Boosting
6. Stacking
7. StackNet

## Averaging ensemble methods
Consider that you want to predict age and you have 2 models. First models had a good prediction in age < 50, but the second model had a good prediction for age >50. So we can do this **(model1 + model2) /2 ** or we can do something like **(model1 x 0.7 + model2 x 0.3)**. But this isn't good enough, there is a better way which is **Prediction of model 1 if age <50, else prediction of model 2**

## Bagging
Means averaging slightly different versions of the same model to improve accuracy
<t> Example of Bagging is random forest
    
### Parameters that control bagging ?
- Changing the seed
- Row (sub) sampling or Bootstrapping (create an artifical dataset)
- Shuffling
- Column (sub) sampling --> choose data for different features !
- Model-specific parameters  --> make ten different model with slightly different regularization
- Number of models (or bags)
- (Optionally) parallelism


## Examples of bagging
BaggingClassifier and BaggingRegressor from sklearn

```python
model = RandomForestRegressor()
bags = 10
seed = 1

# create array object to hold bagged predictions
bagged_prediction = np.zeros(test.shape[0])

# loop for as many times as we want bags
for n in range(0,bags):
    model.set_params(random_state = seed + n) # update seed
    model.fit(train,y) # fit model
    preds = model.predict(test) # predict on test data
    bagged_prediction += preds # add prediction to bagged predictions
    
# take average of predictions
bagged_predictions /= bags```

## Boosting
A form of weighted averaging of models where each model is built sequentially via taking into account the past model performance

### Main boosting types 
1. Weight based
2. Residual based

### Weight based boosting

In [2]:
rows = [[0,0.94,0.27,0.80,0.34,1,0.80,0.20,1.20], [1,0.84,0.79,0.89,0.05,1,0.75,0.25,1.25],[2,0.83,0.11,0.23,0.42,1,0.65,0.35,1.35],
       [3,0.74,0.26,0.03,0.41,0,0.40,0.40,1.40],[4,0.08,0.29,0.76,0.37,0,0.55,0.55,1.55],[5,0.71,0.76,0.43,0.95,1,0.34,0.66,1.66],
       [6,0.08,0.72,0.97,0.04,0,0.02,0.02,1.02]]
df = pd.DataFrame(rows, columns = ['Rownum','x0','x1','x2','x3','y','pred','abs.error','weight'])
df

,Rownum,x0,x1,x2,x3,y,pred,abs.error,weight
0,0,0.94,0.27,0.80,0.34,1,0.80,0.20,1.20
1,1,0.84,0.79,0.89,0.05,1,0.75,0.25,1.25
2,2,0.83,0.11,0.23,0.42,1,0.65,0.35,1.35
3,3,0.74,0.26,0.03,0.41,0,0.40,0.40,1.40
4,4,0.08,0.29,0.76,0.37,0,0.55,0.55,1.55
5,5,0.71,0.76,0.43,0.95,1,0.34,0.66,1.66
6,6,0.08,0.72,0.97,0.04,0,0.02,0.02,1.02


There is various calculation to find the weight, but the purpose are the same. 
<t> What the weight says to the model is 'i want you to put more significance into a certain role'. You can keep repeating this process again calculate a new error based on this error, then calculate new weights --> This is how sequentially add model, maximizing the focus on where the previous models have done wrong.

### Weight based boosting parameters
1. learning rate (shrinkage or eta)
    - predictionN = pred0 + pred1 * eta + ... + predN * eta
2. Number of estimators --> opposite relationship of learning rate
3. Input model - can be anything that accepts weights
4. Sub boosting type :
    - AdaBoost - Good implementation in sklearn (python)
    - LogitBoost - Good implementation in Weka (Java)

### Residual based boosting ---> Most dominant algorithm that take person to win a competition 


In [6]:
rows = [[0,0.94,0.27,0.80,0.34,1,0.80,0.20], [1,0.84,0.79,0.89,0.05,1,0.75,0.25],[2,0.83,0.11,0.23,0.42,1,0.65,0.35],
       [3,0.74,0.26,0.03,0.41,0,0.40,-0.40],[4,0.08,0.29,0.76,0.37,0,0.55,-0.55],[5,0.71,0.76,0.43,0.95,1,0.34,0.66],
       [6,0.08,0.72,0.97,0.04,0,0.02,-0.02]]
df = pd.DataFrame(rows, columns = ['Rownum','x0','x1','x2','x3','y','pred','error'])
df

,Rownum,x0,x1,x2,x3,y,pred,error
0,0,0.94,0.27,0.80,0.34,1,0.80,0.20
1,1,0.84,0.79,0.89,0.05,1,0.75,0.25
2,2,0.83,0.11,0.23,0.42,1,0.65,0.35
3,3,0.74,0.26,0.03,0.41,0,0.40,-0.40
4,4,0.08,0.29,0.76,0.37,0,0.55,-0.55
5,5,0.71,0.76,0.43,0.95,1,0.34,0.66
6,6,0.08,0.72,0.97,0.04,0,0.02,-0.02


Make the error become ur new prediction ! Then predict ur error

In [7]:
rows = [[0,0.94,0.27,0.80,0.34,0.20,0.15,0.80], [1,0.84,0.79,0.89,0.05,0.25,0.20,0.75],[2,0.83,0.11,0.23,0.42,0.35,0.40,0.65],
       [3,0.74,0.26,0.03,0.41,-0.40,-0.30,0.40],[4,0.08,0.29,0.76,0.37,-0.55,-0.20,0.55],[5,0.71,0.76,0.43,0.95,0.66,0.24,0.34],
       [6,0.08,0.72,0.97,0.04,-0.02, -0.01,0.02]]
df = pd.DataFrame(rows, columns = ['Rownum','x0','x1','x2','x3','y', 'newpred','oldpred'])
df

,Rownum,x0,x1,x2,x3,y,newpred,oldpred
0,0,0.94,0.27,0.80,0.34,0.20,0.15,0.80
1,1,0.84,0.79,0.89,0.05,0.25,0.20,0.75
2,2,0.83,0.11,0.23,0.42,0.35,0.40,0.65
3,3,0.74,0.26,0.03,0.41,-0.40,-0.30,0.40
4,4,0.08,0.29,0.76,0.37,-0.55,-0.20,0.55
5,5,0.71,0.76,0.43,0.95,0.66,0.24,0.34
6,6,0.08,0.72,0.97,0.04,-0.02,-0.01,0.02


To predict Rownum=1 we would say:
- Final prediction = 0.75 + 0.20 = 0.95

### Residual based boosting parameters
1. Learning rate (shrinkage or eta)
    - predictionN = pred0 + pred1 * eta + ... + predN * eta, example : Prediction = 0.75 + 0.2*(0.1) = 0.77 (if lr = 0.1)
    - Good way to handle overfit and ensure we don't rely on one model
    
2. Number of estimators
    - Better more, but it takes time
    - If it is too many, your models will have very small contribution (very small lr)
    - Decide these parameters based on cv
    
3. Row(sub) sampling
4. Column(sub) sampling
5. Input model - better be trees.
6. Sub boosting type :
    - Fully gradient based
    - Dart --> Classification problem, it imposes dropout mechanism in order to control the contribution of the trees. This dropout works like regularization. Dropout means, we dispose x% model from y model. 

### Residual based favourite implementations
1. Xgboost
2. Lightgbm
3. H20's GBM
4. Catboost --> strong parameters so you don't need to tuning it
5. Sklearn's GBM --> You can put any sklearn estimator as a base

## Stacking
Means making predictions of a number of models in a hold-out set and then using a different (Meta) model to train on these predictions.

### Methodology
Wolpert in 1992 introduced stacking. It involves:
1. Splitting the train set into two disjoint sets
2. Train several base learners on the first part
3. Make predictions with the base learners on the second (validation) part
4. Using all the predictions from (3) as the input to train a higher level learner


In [8]:
# There are 3 dataset : A (trainset), B(validationset),  C(testset)

# Train algorithm 0 on A and make predictions for B and C and save to B1,C1
# Train algorithm 1 on A and make predictions for B and C and save to B1,C1
# Train algorithm 2 on A and make predictions for B and C and save to B1,C1

rowsB1 = [[0.24,0.72,0.70,0],[0.95,0.25,0.22,1], [0.64,0.80,0.96,0],[0.89,0.58,0.52,0],[0.11,0.20,0.93,1]]
B1 = pd.DataFrame(rowsB1, columns = ['pred0','pred1','pred2','y'])
B1

,pred0,pred1,pred2,y
0,0.24,0.72,0.70,0
1,0.95,0.25,0.22,1
2,0.64,0.80,0.96,0
3,0.89,0.58,0.52,0
4,0.11,0.20,0.93,1


In [9]:
rowsC1 = [[0.50,0.50,0.39,'?'],[0.62,0.59,0.46,'?'], [0.22,0.31,0.54,'?'],[0.90,0.47,0.09,'?'],[0.20,0.09,0.61,'?']]
C1 = pd.DataFrame(rowsC1, columns = ['pred0','pred1','pred2','y'])
C1

,pred0,pred1,pred2,y
0,0.50,0.50,0.39,?
1,0.62,0.59,0.46,?
2,0.22,0.31,0.54,?
3,0.90,0.47,0.09,?
4,0.20,0.09,0.61,?


In [11]:
# Then Train algorithm 3 on B1 and make predictions for C1
rowsC1 = [[0.50,0.50,0.39,'?',0.45],[0.62,0.59,0.46,'?',0.23], [0.22,0.31,0.54,'?',0.99],[0.90,0.47,0.09,'?',0.34],[0.20,0.09,0.61,'?',0.05]]
C1 = pd.DataFrame(rowsC1, columns = ['pred0','pred1','pred2','y','pred3'])
C1

# so pred3, basically our final prediction !

,pred0,pred1,pred2,y,pred3
0,0.50,0.50,0.39,?,0.45
1,0.62,0.59,0.46,?,0.23
2,0.22,0.31,0.54,?,0.99
3,0.90,0.47,0.09,?,0.34
4,0.20,0.09,0.61,?,0.05


```python

# Use 2 base learners
from sklearn.ensemble import RandomForestRegressor #import model
from sklearn.linear_model import LinearRegression #import model
import numpy as np #import numpy for stats
from sklearn.model_selection import train_test_split # split the training data


# split train data in 2 parts, training and validation
training, valid, ytraining, yvalid = train_test_split(train,y,test_size=0.5)
# specify models
model1 = RandomForestRegressor()
model2 = LinearRegression()
# fit models
model1.fit(training,ytraining)
model2.fit(training,ytraining)
# make predictions for validation
preds1= model1.predict(valid)
preds2= model2.predict(valid)
# make predictions for test data
test_preds1 = model1.predict(test)
test_preds2 = model2.predict(test)
# Form a new dataset for valid and test via stacking the predictions
stacked_predictions = np.column_stack((preds1,preds2))
stacked_test_predictions = np.column_stack((test_preds1, test_preds2))
# specify meta model
meta_model = LinearRegression()
# fit meta model on stacked predictions
meta_model.fit(stacked_predictions, yvalid)
# make predictions on the stacked predictions of the test data
final_predictions = meta_model.predict(stacked_test_predictions)```

### Things to be mindful for
1. With time sensitive data - respect time
2. Diversity of model as important as performance. What information new model brings into the table ? Focus on this, even though the new model is weak the meta model still could get benefit.
3. Diversity may come from:
    - Different algorithms (nonlinear and linear)
    - Different input features (different transformation, features, etc)
4. Performance plateauing after N models --> There is a certain number of models when you added to get improvement
5. Meta model is normally modest --> Generally simpler. Lower depth in case if you using randomforest

## StackNet
A scalable meta modelling methodology that utilizes stacking to combine multiple models in a neural network architecture of multiple levels

### StackNet as a neural network
- In a neural network, every node is a simple linear model ( like linear regression) with some non linear transformation
- Instead of a linear model we could use any model


### How to train 
- We cannot use BP (not all models are differentiable)
- We use stacking to link each model/node with target
- To extend to many levels, we can use a Kfold paradigm --> Better use tree model when to train the whole dataset
- No epochs - different connections instead

## TIPS AND TRICKS

### First level tips
1. Diversity based on algorithms:
    - 2-3 Gradient boosted trees (lightgb, xgboost,H20,catboost). Try with bigger depth, low depth, etc
    - 2-3 Neural nets (keras, pytorch) 3 hidden layers, 1 hidden layers --> make slightly different to get new information
    - 1-2 ExtraTrees/RandomForest (sklearn)
    - 1-2 linear models as in logistic/ridge regression, linear svm (sklearn)
    - 1-2 knn models (sklearn)
    - 1 Factorization machine (libfm)
    - 1 svm with nonlinear kernel if size/memory allows (sklearn)
    
2. Diversity based on input data:
    - Categorical features: One hot, label encoding, target encoding
    - Numerical features: outliers, binning, derivatives, percentiles, scalling
    - Interactions: Col1 /+-* Col2, groupby (given a groupby categorical features, calculate the average features), unsupervised (Kmeans, svm, 

### Subsequent level tips
1. Simple (or shallower) algorithms:
    - Gradient boosted trees with small depth (like 2 or 3)
    - Linear models with high regularization
    - Extra Trees --> Don't make them too big
    - Shallow networks (as in 1 hidden layer)
    - knn with BrayCurtis Distance
    - Brute forcing a search for best linear weights based on cv
    
2. Feature Engineering:
    - pairwise differences between meta features
    - row-wise statistics like averages or stds

3. For every 7.5 models in previous level we add 1 in meta
4. Be mindful of target leakage --> Control K value in cv 
 

### Software for Stacking
- StackNet (https://github.com/kaz-Anova/StackNet)
- Stacked ensembles from H20
- Xcessiv (https://github.com/reiinakano/xcessiv)


### Tips about StackNet (Software)
- The parameters sectio. Choose the most important parameters
- Always save your code and re-use it 
- Seek collaborations
- Read forums/kernels